# References

- Memory model: stack frames
- References - pass by reference
  - Avoid heavy copy - `heavy_copy.cpp`
  - Enable passing as super type - `super_type.cpp`
    - Take the opportunity to discuss IO streams and the shell a little more: STDIN, STDOUT, pipe, and redirect
  - Allow modification (this one will take extra time to talk through) - `swap.cpp`
- Const - `reckless.cpp`
- Class: example for each point
  - Heavy copy
  - Super type
  - Modify
  - Const


## The Stack

```c++
int do_more_stuff(int x, int y) { 
    int z = x * y; return x + y + z;
}

int do_stuff(int a, int b) { 
    int c = a + b; int d = a - b; 
    int e = do_more_stuff(c, d); 
    return a + b + e; 
}
```
```c++
int do_it(int first, int second) { 
    return do_stuff(first, second) 
        + do_more_stuff(first, second); 
}

int main() { cout << do_it(4, 5) << endl; }
```

| step | call stack |
| --- |:--- |
| 0 | `main` |
| 1 | `main` , `do_it` |
| 2 | `main` , `do_it`, `do_stuff` |
| 3 | `main` , `do_it`, `do_stuff`, `do_more_stuff` |
| 4 | `main` , `do_it`, `do_stuff` |
| 5 | `main` , `do_it` |
| 6 | `main` , `do_it`, `do_more_stuff` |
| 7 | `main` , `do_it` |
| 8 | `main` |
| 9 | program exits |


| function  | data |  
|---|:---|
| `main` | `do_it return` |
| `do_it` | `do_stuff return`, `do_more_stuff return` |
| `do_stuff` | `int a`, `int b`, `int c`, `int d`, `int e`, `do_more_stuff return` |
| `do_more_stuff` | `int x`, `int y`, `int z` |

Note: depending on the compiler, some optimizations may be performed and your actual usage may be less


## 👷🏻‍♂️ Write out the stack frames at each step of this program

Also, indicate which variables are copied at each step.

```c++
void print_foo(int a) { cout << "foo" << a << endl; }

void print_bar(int b) { cout << "bar" << b << endl; }

void print_foobar(int x, int y) { print_foo(x); print_bar(y); }

void print_quux(int one, int two) { cout << "quux" << endl; print_foobar(one, two); }

void main() { print_foobar(8, 9); print_quux(3, 4); }
```

| step | call stack |
| --- |:--- |
|  0 | `main` |
|  1 | `main` , `print_foobar(x=8, y=9)` |
|  2 | `main` , `print_foobar`, `print_foo(a=x)` |
|  3 | `main` , `print_foobar` |
|  4 | `main` , `print_foobar`, `print_bar(b=y)` |
|  5 | `main` , `print_foobar` |
|  6 | `main` |
|  7 | `main`, `print_quux(one=3, two=4)` |
|  8 | `main`, `print_quux`, `print_foobar(x=one, y=two)` |
|  9 | `main`, `print_quux`, `print_foobar`, `print_foo(a=x)` |
| 10 | `main`, `print_quux`, `print_foobar` |
| 11 | `main`, `print_quux`, `print_foobar`, `print_bar(b=y)` |
| 12 | `main`, `print_quux`, `print_foobar` |
| 13 | `main`, `print_quux` |
| 14 | `main` |
| 15 | program exits |


## `heavy_copy.cpp`

`make heavy && ./heavy really_big_text.txt 100`

### What is happening in `heavy_copy.cpp`?

When you **pass by reference**, instead of copying the data at each step, the function *references* the data in the other stack. No data is copied.

## `super_type.cpp`

- note how `istream` can except both `cin` and `ifstream`
- talk about **abstract base classes**
  - you can't instantiate them
  - but you can treat different children classes as the same parent type
- talk about interfaces
  - the `bullets` method doesn't need to know how or where your stream is coming from
  - it just needs to know that the thing behaves like an input stream

`make super && ./super text.txt`

`make super && ./super`

`cat text.txt | ./super > bulletted_text.txt`

`cat text.txt | ./super | ./super | ./super`

- See how it echos with bullets line-by-line.
- Use <kbd>ctrl</kbd> + <kbd>D</kbd> to close the input stream.
- Demonstrate how to use the shell pipe `|` to pass data to `./super` via STDIN
  - show `echo`, `cat`, and `head`
  - `cat text.txt | ./super`
- Demonstrate how to use the STDOUT redirect `>` to pass STDOUT to a file
  - `cat text.txt | ./super > bulletted_text.txt`

## `swap.cpp`

- `where.h` contains a macro `WHERE` that prints the memory address of a variable
- explain: no new memory is allocated for `&first` and `&second`
  - rather, the variable names `first` and `second` are used to reference the same memory address as `a` and `b`
  - see the addresses `WHERE` prints
- remove the `&` from the method signature
  - see how the addresses change - we are now swapping different memory and the original memory stays unchanged
  
`make swap && ./swap 7 8`

## 🤔 `get_file_contents`

If we modified the `get_file_contents` method from `heavy_copy.cpp` to have the signature 
```c++
void get_file_contents(string filename, string &contents);
```

how would it affect the number of times the file contents are copied?

## `reckless.cpp`

- the `const_demo_framing.cpp`, `const_demo_telling.cpp`, and `const_demo_punchline.cpp` sequence outline the sequence of changes to make this point
- `make reckless && ./reckless 4 'this is some text'`
- now irresponsibly add something like the following to the method `reckless_with_your_data`
  ```c++
  number = 2;
  phrase = "You keep saying those words. I do not think they are what you think they are.";
  ```
- `make reckless && ./reckless 4 'this is some text'`
  - `reckless_with_your_data` changed the values!
- add the `const` qualifier in the method signature -> notice the compiler error
- moral: `const` keeps functions from changes the values of our references

#### `const_demo_framing.cpp`

#### `const_demo_telling.cpp`

#### `const_demo_punchline.cpp`

## Key Ideas
- Passing data *by reference* using the `&` modifier allows multiple functions to operate on the same memory location
  - Avoids unnecessary data copies
  - Allows subtype objects to be passed as supertypes
  - Delegates the modification of data to another function
  - Use `const` to make a reference readonly when the data should **not** change